# Analyse des sentiments / RNN

In [1]:
# importation des librairies
import numpy as np

In [2]:
w =np.random.randn(3,4) # la valeur magique (poids)
w

array([[ 1.91646014e+00,  3.32090061e-05, -9.92933612e-01,
        -3.90390348e-01],
       [ 6.32737267e-02,  7.57153075e-01,  6.92512433e-01,
        -1.35460664e+00],
       [ 1.41670205e-01,  1.37350980e+00,  2.31704735e-01,
         2.44302641e-01]])

In [3]:
b= np.random.randn(4) # bias (regulateur)
b

array([ 0.98922637,  1.15375727, -0.4279091 ,  0.14978265])

In [4]:
X= np.random.randn(3)
X

array([ 0.87115722, -0.20040827,  0.08048825])

In [5]:
def sigmoid(x):
  z= 1/(1+np.exp(-x))
  return z

# Traitements
  * _Elimination des stop word_ : Eliminer les articles, les pronoms, adverbes, ponctuations ... pour éviter au modele d'apprendre plus sur cela
  * _Encodage_ : on cree notre vocabulaire (NLP natural language preprocessing) cad l'ensemble des mots qui constituent notre document ou tout nos commentaires ou notre test d'analyse des sentiments, sans repetition. </br>
    - puis on vas le remplacer selon un encodage
    - nous aurons donc des données labelisées

In [11]:
!pip3 install keras --quiet
# import tensorflow as tf
from tensorflow.keras.datasets import imdb

In [9]:
(X_train,y_train),(X_test,y_test)=imdb.load_data(num_words=10000)

X_train.shape, y_train.shape, X_test.shape,y_test.shape

17473536/17464789 [==============================] - 0s 0us/step


((25000,), (25000,), (25000,), (25000,))

In [12]:
len(X_train[1]) # les commentaires n'ont pas la meme taille car il s'agit des avis differents

189

In [14]:
from tensorflow.keras.preprocessing import sequence

In [15]:
# il vas prendre les commentaires, et check tout les commentaires qui n'atteingnet pas la taille de 1000, value signifie taille vide, 1 constitue deja un mot
# chaque commentaire aura donc 1000 mot
X_train = sequence.pad_sequences(X_train, value = 0, padding="post", maxlen=1000) 

In [ ]:
X_train[0]

In [17]:
X_test = sequence.pad_sequences(X_test, value = 0, padding="post", maxlen=1000) 

# Creation du modele RNN

In [18]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [19]:
# on creer l'architecture avec les couches RNN d'abord, puis apres les couches denses (qui utilisent des vecteurs)
def model_rnn(vocab_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size,output_dim=32,input_length=1000)) # 32 = 32 sequences pour le Simple RNN, ceci cree la couche qui transforme les données en des sequences, imput_dim = taille du vocabulaire
  model.add(SimpleRNN(32, activation='relu', return_sequences=True)) # couche RNN qui renvois des sequences, 32 = 32 neurones. Nombres des sequences doivent etre egale au nombre des neurones a recevoir
  model.add(SimpleRNN(12, activation='relu', return_sequences=False)) # couche RNN qui renvoie des vecteurs
  model.add(Dense(1, activation='sigmoid')) # il renvois un sigmoid
  return model

In [20]:
model = model_rnn(10000)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 32)          320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 1000, 32)          2080      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 16)                784       
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 322,881
Trainable params: 322,881
Non-trainable params: 0
_________________________________________________________________


In [23]:
# on entraine, avat on compile
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']) # adam = function d'optimization pour atteindre le bon resultat le plus rapidement, binary_crossentropy = 

In [24]:
model.fit(X_train,y_train,batch_size=32,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
782/782 [==============================] - 467s 594ms/step - loss: 0.6934 - accuracy: 0.4975 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/5
782/782 [==============================] - 464s 593ms/step - loss: 114392236032.0000 - accuracy: 0.5035 - val_loss: 0.6932 - val_accuracy: 0.5002
Epoch 3/5
782/782 [==============================] - 464s 594ms/step - loss: 0.6928 - accuracy: 0.5030 - val_loss: 0.6931 - val_accuracy: 0.4999
Epoch 4/5
782/782 [==============================] - 461s 589ms/step - loss: 0.6921 - accuracy: 0.5011 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 5/5
782/782 [==============================] - 461s 590ms/step - loss: 0.6911 - accuracy: 0.5045 - val_loss: 0.6936 - val_accuracy: 0.5000
